## Bayesian Personalized ranking model for movie recommendation ranking

In [1]:
import numpy as np
import pandas as pd
from os import path
from collections import OrderedDict
from tqdm import tqdm
from typing import Dict

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Embedding, Flatten, Input, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score

from recommenders.datasets import movielens

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
tf.__version__

'2.8.0'

In [3]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
DIR_DATA = 'data'
DIR_MODEL = 'models'

### Get data

In [5]:
data = movielens.load_pandas_df(
    size='100k',
    header=['user', 'movie', 'rating', 'Timestamp'],
    title_col='title'
)

data.loc[:, 'rating'] = data['rating'].astype(np.int32)
data['id'] = data.apply(lambda x: str(x['user'])+"_"+str(x['movie']), axis=1)
df_full = data.loc[:, ['user', 'movie', 'rating', 'id']]
df_full.head()

100%|██████████| 4.81k/4.81k [00:08<00:00, 537KB/s]


,user,movie,rating,id
0,196,242,3,196_242
1,63,242,3,63_242
2,226,242,5,226_242
3,154,242,3,154_242
4,306,242,5,306_242


### Build the references

I'm planning to use the `Embedding` layer, so I need to link real ids of the users and movies to the order ones.

In [6]:
sorted(df_full.movie.unique())[-10:]

[1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682]

In [7]:
sorted(df_full.user.unique())[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [8]:
unique_users = df_full.user.unique()
user_ids = dict(zip(unique_users, np.arange(unique_users.shape[0], dtype=np.int32)))

unique_movies = df_full.movie.unique()
movie_ids = dict(zip(unique_movies, np.arange(unique_movies.shape[0], dtype=np.int32)))

In [9]:
df_full['user_id'] = df_full.user.apply(lambda u: user_ids[u])
df_full['movie_id'] = df_full.movie.apply(lambda m: movie_ids[m])

df_full.head(1)

,user,movie,rating,id,user_id,movie_id
0,196,242,3,196_242,0,0


### Train/test split

Here the main idea is to extract some movies for users who have a big amount of positive reviews into the test subtest. I extract 2 movies for each user who have more than 20 positive reviews. This test subset won't be used during training, but these movies should appear in the top recommendations for each user accordingly.

#### Test subset

In [10]:
tmp_test = df_full[df_full.rating > 4]
tmp_test = tmp_test.groupby('user').movie.count().reset_index()
tmp_test.shape

(928, 2)

In [11]:
conditions = (df_full.user.isin(tmp_test[tmp_test.movie > 20].user)) & (df_full.rating > 4)
df_test = df_full[conditions].groupby('user').head(2).reset_index()

del df_test['index']
df_test.shape

(698, 6)

In [12]:
ground_truth_test = df_test.groupby('user_id').movie_id.agg(list).reset_index()

ground_truth_test.head(1)

,user_id,movie_id
0,5,"[10, 12]"


#### Training subset

In [13]:
df_train = pd.concat([df_full, df_test]).drop_duplicates(keep=False)
df_train.shape

(99302, 6)

In [14]:
# The assumption is that the recommendations should as many as possible high ranked movies 
# that a specific user has already watched.

ground_truth_train = df_train[df_train.rating > 3].groupby('user_id').movie_id.agg(list).reset_index()

ground_truth_train.head(1)

,user_id,movie_id
0,0,"[83, 86, 92, 221, 289, 302, 329, 377, 380, 389..."


### Building triplets

Bayers Personalized Ranking requires for the training a triplet of the user, positive item and negative item. For each user, I create a pair of each positive ranked movie (the rank is higher than 3) with all negative movies (the rank is equal  3 and lower than).

In [15]:
df_triplest = pd.DataFrame(columns=['user_id', 'positive_m_id', 'negative_m_id'])
df_triplest

,user_id,positive_m_id,negative_m_id


In [16]:
%%time

data = []
users_without_data = []

for user_id in tqdm(df_train.user_id.unique()):
    positive_movies = df_train[(df_train.user_id == user_id) & (df_train.rating > 3)].movie_id.values
    negative_movies = df_train[(df_train.user_id == user_id) & (df_train.rating <= 3)].movie_id.values

    if negative_movies.shape[0] == 0 or positive_movies.shape[0] == 0:
        users_without_data.append(user_id)
        continue


    for positive_movie in positive_movies:
        for negative_movie in negative_movies:
            data.append({'user_id': user_id, 'positive_m_id': positive_movie, 'negative_m_id': negative_movie})

df_triplest = df_triplest.append(data, ignore_index=True)
# df_triplest.to_csv(path.join(DIR_DATA, 'triplets.csv'), index=False)

100%|██████████| 943/943 [00:03<00:00, 294.58it/s]


CPU times: user 8 s, sys: 938 ms, total: 8.94 s
Wall time: 9.06 s


In [17]:
df_triplest.shape, df_train.shape

((4349304, 3), (99302, 6))

In [18]:
df_triplest

,user_id,positive_m_id,negative_m_id
0,0,83,0
1,0,83,10
2,0,83,179
3,0,83,291
4,0,83,321
...,...,...,...
4349299,942,1116,360
4349300,942,1116,491
4349301,942,1116,570
4349302,942,1116,959


### BPR NN

In [19]:
num_users = unique_users.shape[0]
num_items = unique_movies.shape[0]

num_users, num_items

(943, 1682)

In [20]:
unique_movie_ids = list(df_full.movie_id.unique())

### Build a model

In [21]:
def bpr_predict(model: Model, user_id: int, item_ids: list, user_layer='user_embedding', item_layer='item_embedding'):
    """
    Predict by multiplication user vector by item matrix
    
    :return: list of the scores
    """
    user_vector = model.get_layer(user_layer).get_weights()[0][user_id]
    item_matrix = model.get_layer(item_layer).get_weights()[0][item_ids]

    scores = (np.dot(user_vector, item_matrix.T))

    return scores

In [22]:
def ranking(model, user_id, item_ids):
    item_scores = [bpr_predict(model, user_id, i) for i in item_ids]
    res_df = pd.DataFrame({'item_id': item_ids, 'score': item_scores}).sort_values(by='score', ascending=False)
    return res_df
    

In [23]:
@tf.function
def identity_loss(_, y_pred):
    return tf.math.reduce_mean(y_pred)

In [24]:
@tf.function
def bpr_triplet_loss(X: dict):
    """
    Calculate triplet loss - as higher the difference between positive interactions
    and negative interactions as better

    :param X: X contains the user input, positive item input, negative item input
    :return:
    """
    positive_item_latent, negative_item_latent, user_latent = X

    positive_interactions = tf.math.reduce_sum(tf.math.multiply(user_latent, positive_item_latent), axis=-1, keepdims=True)
    negative_interactions = tf.math.reduce_sum(tf.math.multiply(user_latent, negative_item_latent), axis=-1, keepdims=True)

    return tf.math.subtract(tf.constant(1.0), tf.sigmoid(tf.math.subtract(positive_interactions, negative_interactions)))

In [25]:
def out_shape(shapes):
    return shapes[0]
    

def build_model(num_users: int, num_items: int, latent_dim: int) -> Model:
    """
    Build a model for Bayesian personalized ranking

    :param num_users: a number of the unique users
    :param num_items: a number of the unique movies
    :param latent_dim: vector length for the latent representation
    :return: Model
    """
    user_input = Input((1,), name='user_input')

    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')
    # One embedding layer is shared between positive and negative items
    item_embedding_layer = Embedding(num_items, latent_dim, name='item_embedding', input_length=1)

    positive_item_embedding = Flatten()(item_embedding_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_embedding_layer(negative_item_input))

    user_embedding = Embedding(num_users, latent_dim, name='user_embedding', input_length=1)(user_input)
    user_embedding = Flatten()(user_embedding)

    triplet_loss = Lambda(bpr_triplet_loss, output_shape=out_shape)([positive_item_embedding,
                                                             negative_item_embedding,
                                                             user_embedding])

    model = Model(inputs=[positive_item_input, negative_item_input, user_input], outputs=triplet_loss)

    return model

In [26]:
latent_dim = 350
batch_size = 256
num_epochs = 1
lr = 0.001

model = build_model(num_users, num_items, latent_dim)
model.compile(loss=identity_loss, optimizer=Adam(learning_rate=lr))

2022-04-27 16:02:45.016140: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

print('Total number of parameters: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable number of parameters: {:,}'.format(trainable_count))
print('Non-trainable number of parameters: {:,}'.format(non_trainable_count))

print('Training data length: {:,}'.format(df_triplest.shape[0]))

Total number of parameters: 918,750.0
Trainable number of parameters: 918,750
Non-trainable number of parameters: 0.0
Training data length: 4,349,304


In [28]:
%%time

X = {
    'user_input': tf.convert_to_tensor(df_triplest.user_id, dtype=tf.int32),
    'positive_item_input': tf.convert_to_tensor(df_triplest.positive_m_id, dtype=tf.int32),
    'negative_item_input': tf.convert_to_tensor(df_triplest.negative_m_id, dtype=tf.int32)
}

model.fit(X, 
          tf.ones(df_triplest.shape[0]), 
          batch_size=batch_size,
          epochs=num_epochs)

# model.save(path.join(DIR_MODEL, 'model.h5'))

16990/16990 [==============================] - 104s 6ms/step - loss: 0.0243
CPU times: user 6min 34s, sys: 1min 7s, total: 7min 41s
Wall time: 1min 44s


### Evaluation

In [29]:
# predict for user_id : 0 and items_ids : [12, 232, 54, 34, 98]
res_df = ranking(model, 0, [12, 232, 54, 34, 98])
res_df

,item_id,score
1,232,2.795310
2,54,0.254559
3,34,-0.120102
0,12,-0.808267
4,98,-3.324351


In [30]:
def full_auc(model: Model, ground_truth: Dict[int, list], items: list) -> float:
    """
    Measure AUC for model and ground truth for all items
    
    :param model: 
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :return: AUC
    """

    number_of_items = len(items)
    scores = []

    for user_id, true_item_ids in ground_truth:
        predictions = bpr_predict(model, user_id, items)
        grnd = np.zeros(number_of_items, dtype=np.int32)

        for p in true_item_ids:
            index = items.index(p)
            grnd[index] = 1

        if true_item_ids:
            scores.append(roc_auc_score(grnd, predictions))

    return sum(scores) / len(scores)

In [31]:
def mean_average_precision_k(model: Model, 
                           ground_truth: Dict[int, list], 
                           items: list, 
                           k=100) -> float:
    """
    Calculate mean eavarage precission per user
    
    :param model: 
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :param k: top N recommendations per user
    :return: mean eavarage precission
    """
    scores = []

    for user, actual in ground_truth:
        predictions = bpr_predict(model, user, items)
        predictions = dict(zip(items, predictions))
        predictions = sorted(predictions.items(), key=lambda kv: kv[1], reverse=True)[:k]
        predictions = list(OrderedDict(predictions).keys())

        score = 0.0
        num_hits = 0.0

        for i, p in enumerate(predictions):
            if p in actual:
                num_hits += 1.0
                score += num_hits / (i + 1.0)

        score = score / min(len(actual), k)
        scores.append(score)

    return np.mean(scores)

#### Train

In [32]:
print(f'AUC train: {full_auc(model, ground_truth_train.values, unique_movie_ids)}')

AUC train: 0.9607443761926987


In [38]:
print(f'Mean average precision train: {mean_average_precision_k(model, ground_truth_train.values, unique_movie_ids)}')

Mean average precision train: 0.46296911974689686


#### Test

In [71]:
print(f'AUC test: {full_auc(model, ground_truth_test.values, unique_movie_ids)}')

AUC test: 0.7855946544298201


In [72]:
print(f'Mean average precision test: {mean_average_precision_k(model, ground_truth_test.values, unique_movie_ids)}')

Mean average precision test: 0.014455187878782139
